# Обнаружение страхового мошенничества с использованием машинного обучения

## Описание проекта
Проект направлен на обнаружение потенциального страхового мошенничества в данных о страховых убытках. В проекте используются различные методы машинного обучения для анализа данных и предсказания вероятности мошенничества.

## Цели проекта
- Проведение анализа и предварительной обработки данных.
- Обучение модели машинного обучения для предсказания вероятности мошенничества.
- Оптимизация процесса проверки убытков, с минимизацией количества проверок, при этом снижением количества выявленных мошеннических убытков не более чем на 20%.

## Данные
Данные представляют собой набор записей о страховых убытках, связанных с автомобильными авариями. В данных содержится информация о типе убытка, дате регистрации претензии, сумме убытка и других деталях.

In [1]:
from google.colab import drive

drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


In [2]:
file_path = "/content/MyDrive/MyDrive/study_datasets/Тестовое задание 2024-04.csv"

In [3]:
import chardet
import pandas as pd

with open('/content/MyDrive/MyDrive/study_datasets/Тестовое задание 2024-04.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read(10000))

insurance_data = pd.read_csv(file_path, encoding=result['encoding'], sep=';')

In [4]:
insurance_data.head()

,тип убытка,Номер убытка,Номер претензии,Дата события,Время ДТП,Место ДТП,Вид извещения о событии,Характер события,Оформлено сотрудниками ГИБДД,Адрес ДТП: Страна,...,Риск,Объект,Основание для претензии,Характер ущерба,Форма воозмещения,Основание для регресса,Дата предотсавления полного пакета документов,сумма,проверка,мошенник
0,У,У-000-01799744/16,У-000-01799744/16/3,20.06.2012,13:00:00,Автомагистраль,Заявление,ДТП с участием 2-х ТС с пострадавшими,ДА,Россия,...,Риски ОСАГО,Man TGA,Судебное решение,Смерть в результате ДТП,Денежная,NaN,NaN,12 500,Да,0
1,У,У-000-01799744/16,У-000-01799744/16/1,20.06.2012,13:00:00,Автомагистраль,Заявление,ДТП с участием 2-х ТС с пострадавшими,ДА,Россия,...,Риски ОСАГО,Man TGA,Судебное решение,Смерть в результате ДТП,Денежная,NaN,NaN,12 500,Да,0
2,У,У-000-01799744/16,У-000-01799744/16/2,20.06.2012,13:00:00,Автомагистраль,Заявление,ДТП с участием 2-х ТС с пострадавшими,ДА,Россия,...,Риски ОСАГО,Man TGA,Заявление от потерпевшего,Прочие повреждения,Денежная,NaN,19.10.2016,120 000,Да,0
3,ПВУ,ПВУ-991-077356/15,ПВУ-991-077356/15/1,17.03.2013,10:10:00,В пределах дорог (кроме автомагистралей),Заявление,ДТП с участием 2-х ТС без пострадавших,ДА,Россия,...,Риски ОСАГО,Porsche Cayenne S <K438KH177>,Заявление от потерпевшего,Прочие повреждения,NaN,NaN,01.04.2015,33 788,NaN,NaN
4,У,У-991-01781532/16,У-991-01781532/16/1,26.03.2013,18:00:00,В пределах дорог (кроме автомагистралей),Заказное письмо,ДТП с участием более 2-х ТС с пострадавшими,ДА,Россия,...,Риски ОСАГО,Chevrolet Niva,Заявление от потерпевшего,Прочие повреждения,Денежная,NaN,18.03.2016,120 000,Да,0


In [5]:
insurance_data.describe()

,тип убытка,Номер убытка,Номер претензии,Дата события,Время ДТП,Место ДТП,Вид извещения о событии,Характер события,Оформлено сотрудниками ГИБДД,Адрес ДТП: Страна,...,Риск,Объект,Основание для претензии,Характер ущерба,Форма воозмещения,Основание для регресса,Дата предотсавления полного пакета документов,сумма,проверка,мошенник
count,14491,14491,14491,14491,14491,14491,14491,14491,14491,14491,...,14491,14491,14488,14486,11597,436,10971,13243,6434,6434
unique,2,10748,14476,1305,803,10,5,16,2,1,...,1,9024,7,20,2,12,847,9705,1,3
top,ПВУ,У-991-01850093/17,У-991-01742715/15/2,22.01.2016,13:00:00,В пределах дорог (кроме автомагистралей),Заявление,ДТП с участием 2-х ТС без пострадавших,ДА,Россия,...,Риски ОСАГО,Kia Rio 0 <>,Заявление от потерпевшего,Прочие повреждения,Денежная,"Алкогольное, наркотическое или иное токсическо...",23.01.2017,400 000,Да,0
freq,12320,9,2,40,226,11006,11223,12498,11870,14491,...,14491,102,13537,12937,11456,148,35,570,6434,5828


In [6]:
insurance_data = insurance_data.rename(columns={
    "Дата предотсавления полного пакета документов": "Дата предоставления полного пакета документов",
    "Форма воозмещения": "Форма возмещения"
})

# Исправляем пропущенные значения (Missing Values)


In [7]:
missing_values = insurance_data.isnull().sum()
missing_values

тип убытка                                           0
Номер убытка                                         0
Номер претензии                                      0
Дата события                                         0
Время ДТП                                            0
Место ДТП                                            0
Вид извещения о событии                              0
Характер события                                     0
Оформлено сотрудниками ГИБДД                         0
Адрес ДТП: Страна                                    0
Дата регистрации претензии                           0
Договор                                              0
Риск                                                 0
Объект                                               0
Основание для претензии                              3
Характер ущерба                                      5
Форма возмещения                                  2894
Основание для регресса                           14055
Дата предо

In [8]:
most_frequient_complaints_data = insurance_data["Основание для претензии"].value_counts().idxmax()
insurance_data["Основание для претензии"].fillna(most_frequient_complaints_data, inplace=True)

most_frequient_damage = insurance_data["Характер ущерба"].value_counts().idxmax()
insurance_data["Характер ущерба"].fillna(most_frequient_damage, inplace=True)

most_frequient_refund_form = insurance_data["Форма возмещения"].value_counts().idxmax()
insurance_data["Форма возмещения"].fillna(most_frequient_refund_form, inplace=True)

most_frequient_grounds_for_recourse = insurance_data["Основание для регресса"].value_counts().idxmax()
insurance_data["Основание для регресса"].fillna(most_frequient_grounds_for_recourse, inplace=True)

insurance_data['Дата предоставления полного пакета документов'].fillna(method='bfill', inplace=True)
insurance_data['Дата предоставления полного пакета документов'].fillna(method='ffill', inplace=True)

def remove_spaces(value):
  if isinstance(value, str):
    return value.replace(" ", "")
  else:
    return value

insurance_data["сумма"] = insurance_data["сумма"].apply(remove_spaces)
insurance_data['сумма'] = pd.to_numeric(insurance_data['сумма'], errors='coerce')
mean_sum_value = insurance_data["сумма"].mean()
insurance_data["сумма"].fillna(mean_sum_value, inplace=True)

def transform_element(element):
  if isinstance(element, str):
    mapping = {"да": 1, "нет": 0}
    return mapping.get(element.lower(), element)
  elif pd.isnull(element):
    return 0
  else:
    return element

insurance_data["проверка"] = insurance_data["проверка"].apply(transform_element)
insurance_data["мошенник"] = insurance_data["мошенник"].apply(transform_element)

In [9]:
missing_values = insurance_data.isnull().sum()
missing_values

тип убытка                                       0
Номер убытка                                     0
Номер претензии                                  0
Дата события                                     0
Время ДТП                                        0
Место ДТП                                        0
Вид извещения о событии                          0
Характер события                                 0
Оформлено сотрудниками ГИБДД                     0
Адрес ДТП: Страна                                0
Дата регистрации претензии                       0
Договор                                          0
Риск                                             0
Объект                                           0
Основание для претензии                          0
Характер ущерба                                  0
Форма возмещения                                 0
Основание для регресса                           0
Дата предоставления полного пакета документов    0
сумма                          

In [10]:
insurance_data.head()

,тип убытка,Номер убытка,Номер претензии,Дата события,Время ДТП,Место ДТП,Вид извещения о событии,Характер события,Оформлено сотрудниками ГИБДД,Адрес ДТП: Страна,...,Риск,Объект,Основание для претензии,Характер ущерба,Форма возмещения,Основание для регресса,Дата предоставления полного пакета документов,сумма,проверка,мошенник
0,У,У-000-01799744/16,У-000-01799744/16/3,20.06.2012,13:00:00,Автомагистраль,Заявление,ДТП с участием 2-х ТС с пострадавшими,ДА,Россия,...,Риски ОСАГО,Man TGA,Судебное решение,Смерть в результате ДТП,Денежная,"Алкогольное, наркотическое или иное токсическо...",19.10.2016,12500.0,1,0
1,У,У-000-01799744/16,У-000-01799744/16/1,20.06.2012,13:00:00,Автомагистраль,Заявление,ДТП с участием 2-х ТС с пострадавшими,ДА,Россия,...,Риски ОСАГО,Man TGA,Судебное решение,Смерть в результате ДТП,Денежная,"Алкогольное, наркотическое или иное токсическо...",19.10.2016,12500.0,1,0
2,У,У-000-01799744/16,У-000-01799744/16/2,20.06.2012,13:00:00,Автомагистраль,Заявление,ДТП с участием 2-х ТС с пострадавшими,ДА,Россия,...,Риски ОСАГО,Man TGA,Заявление от потерпевшего,Прочие повреждения,Денежная,"Алкогольное, наркотическое или иное токсическо...",19.10.2016,120000.0,1,0
3,ПВУ,ПВУ-991-077356/15,ПВУ-991-077356/15/1,17.03.2013,10:10:00,В пределах дорог (кроме автомагистралей),Заявление,ДТП с участием 2-х ТС без пострадавших,ДА,Россия,...,Риски ОСАГО,Porsche Cayenne S <K438KH177>,Заявление от потерпевшего,Прочие повреждения,Денежная,"Алкогольное, наркотическое или иное токсическо...",01.04.2015,33788.0,0,0
4,У,У-991-01781532/16,У-991-01781532/16/1,26.03.2013,18:00:00,В пределах дорог (кроме автомагистралей),Заказное письмо,ДТП с участием более 2-х ТС с пострадавшими,ДА,Россия,...,Риски ОСАГО,Chevrolet Niva,Заявление от потерпевшего,Прочие повреждения,Денежная,"Алкогольное, наркотическое или иное токсическо...",18.03.2016,120000.0,1,0


# Преобразование категориальных переменных с помощью Label Encoding

В этом блоке происходит преобразование категориальных переменных в данных в числовые значения с использованием метода Label Encoding. Это необходимо, чтобы модели машинного обучения могли работать с этими данными, так как они могут обрабатывать только числовые значения.

Для выполнения этого преобразования используется LabelEncoder из библиотеки sklearn.preprocessing. Для каждого столбца создается экземпляр LabelEncoder, который обучается на данных из этого столбца и применяется для преобразования данных. Каждый экземпляр LabelEncoder сохраняется в словаре для последующего использования для обратного преобразования, если это будет необходимо.

In [11]:
from sklearn.preprocessing import LabelEncoder


label_encoders = {} # для хранения экземпляров LabelEncoder для каждого столбца
insurance_data_encoded = insurance_data.copy() # copy

# Применяем LabelEncoder ко всем столбцам, кроме числовых
for col in insurance_data.columns:
    if insurance_data[col].dtype == 'object' or insurance_data[col].dtype == 'bool':

        # Преобразуем все значения в столбце в строки
        insurance_data_encoded[col] = insurance_data_encoded[col].astype(str)

        # Создаем и обучаем LabelEncoder для этого столбца
        le = LabelEncoder()
        insurance_data_encoded[col] = le.fit_transform(insurance_data_encoded[col])

        label_encoders[col] = le # Сохраняем LabelEncoder в словаре

        print(f"Столбец '{col}' был обработан.")

Столбец 'тип убытка' был обработан.
Столбец 'Номер убытка' был обработан.
Столбец 'Номер претензии' был обработан.
Столбец 'Дата события' был обработан.
Столбец 'Время ДТП' был обработан.
Столбец 'Место ДТП' был обработан.
Столбец 'Вид извещения о событии' был обработан.
Столбец 'Характер события' был обработан.
Столбец 'Оформлено сотрудниками ГИБДД' был обработан.
Столбец 'Адрес ДТП: Страна' был обработан.
Столбец 'Дата регистрации претензии' был обработан.
Столбец 'Договор' был обработан.
Столбец 'Риск' был обработан.
Столбец 'Объект' был обработан.
Столбец 'Основание для претензии' был обработан.
Столбец 'Характер ущерба' был обработан.
Столбец 'Форма возмещения' был обработан.
Столбец 'Основание для регресса' был обработан.
Столбец 'Дата предоставления полного пакета документов' был обработан.
Столбец 'мошенник' был обработан.


In [12]:
column_names = insurance_data.columns
print(column_names)

Index(['тип убытка', 'Номер убытка', 'Номер претензии', 'Дата события',
       'Время ДТП', 'Место ДТП', 'Вид извещения о событии', 'Характер события',
       'Оформлено сотрудниками ГИБДД', 'Адрес ДТП: Страна',
       'Дата регистрации претензии', 'Договор', 'Риск', 'Объект',
       'Основание для претензии', 'Характер ущерба', 'Форма возмещения',
       'Основание для регресса',
       'Дата предоставления полного пакета документов', 'сумма', 'проверка',
       'мошенник'],
      dtype='object')


In [13]:
column_names = insurance_data_encoded.columns
print(column_names)

Index(['тип убытка', 'Номер убытка', 'Номер претензии', 'Дата события',
       'Время ДТП', 'Место ДТП', 'Вид извещения о событии', 'Характер события',
       'Оформлено сотрудниками ГИБДД', 'Адрес ДТП: Страна',
       'Дата регистрации претензии', 'Договор', 'Риск', 'Объект',
       'Основание для претензии', 'Характер ущерба', 'Форма возмещения',
       'Основание для регресса',
       'Дата предоставления полного пакета документов', 'сумма', 'проверка',
       'мошенник'],
      dtype='object')


In [14]:
label_encoders

{'тип убытка': LabelEncoder(),
 'Номер убытка': LabelEncoder(),
 'Номер претензии': LabelEncoder(),
 'Дата события': LabelEncoder(),
 'Время ДТП': LabelEncoder(),
 'Место ДТП': LabelEncoder(),
 'Вид извещения о событии': LabelEncoder(),
 'Характер события': LabelEncoder(),
 'Оформлено сотрудниками ГИБДД': LabelEncoder(),
 'Адрес ДТП: Страна': LabelEncoder(),
 'Дата регистрации претензии': LabelEncoder(),
 'Договор': LabelEncoder(),
 'Риск': LabelEncoder(),
 'Объект': LabelEncoder(),
 'Основание для претензии': LabelEncoder(),
 'Характер ущерба': LabelEncoder(),
 'Форма возмещения': LabelEncoder(),
 'Основание для регресса': LabelEncoder(),
 'Дата предоставления полного пакета документов': LabelEncoder(),
 'мошенник': LabelEncoder()}

In [19]:
print(insurance_data_encoded['мошенник'].value_counts())

мошенник
0    13885
1      606
Name: count, dtype: int64


In [20]:
!pip install imbalanced-learn

In [21]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X, y)

# Проверка баланса классов
print(y_res.value_counts())

мошенник
0    13885
1    13885
Name: count, dtype: int64


In [22]:
X_res

,тип убытка,Номер убытка,Номер претензии,Дата события,Время ДТП,Место ДТП,Вид извещения о событии,Характер события,Оформлено сотрудниками ГИБДД,Адрес ДТП: Страна,...,Договор,Риск,Объект,Основание для претензии,Характер ущерба,Форма возмещения,Основание для регресса,Дата предоставления полного пакета документов,сумма,проверка
0,1,9751,12348,834,154,0,2,7,0,0,...,10,0,3859,6,14,0,0,515,12500.000000,1
1,1,9751,12346,834,154,0,2,7,0,0,...,10,0,3859,6,14,0,0,515,12500.000000,1
2,1,9751,12347,834,154,0,2,7,0,0,...,10,0,3859,4,10,0,0,515,120000.000000,1
3,0,4808,6446,693,34,1,2,4,0,0,...,5,0,6027,4,10,0,0,7,33788.000000,0
4,1,10631,14199,1079,373,1,0,9,0,0,...,13,0,955,4,10,0,0,469,120000.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27765,0,2672,3874,228,510,1,3,4,0,0,...,2619,0,6014,4,2,0,0,218,120906.404997,1
27766,0,1166,1803,865,399,1,2,4,0,0,...,8713,0,8910,4,10,0,0,559,31143.922232,1
27767,0,5151,6818,520,163,1,2,4,0,0,...,829,0,4244,4,10,0,0,310,400000.000000,1
27768,0,662,1119,748,247,1,1,4,0,0,...,4789,0,2629,4,10,0,0,93,76941.445296,1


In [23]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Разделение данных
X_res_train, X_res_test, y_res_train, y_res_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)

# Выбор модели
model_res = RandomForestClassifier()

# Обучение модели
model_res.fit(X_res_train, y_res_train)

# Оценка модели
predictions_res = model_res.predict(X_res_test)
print(classification_report(y_res_test, predictions_res))

              precision    recall  f1-score   support

           0       0.99      0.96      0.98      2779
           1       0.96      0.99      0.98      2775

    accuracy                           0.98      5554
   macro avg       0.98      0.98      0.98      5554
weighted avg       0.98      0.98      0.98      5554



In [26]:
# Создание DataFrame из сбалансированных данных
resampled_data = pd.DataFrame(X_res, columns=X.columns)
resampled_data['мошенник'] = y_res

# Сравнение исходных и сбалансированных данных
original_data = insurance_data_encoded[insurance_data_encoded['мошенник'] == 1]
synthetic_data = resampled_data[~resampled_data.index.isin(original_data.index)]

print(synthetic_data)

       тип убытка  Номер убытка  Номер претензии  Дата события  Время ДТП  \
0               1          9751            12348           834        154   
1               1          9751            12346           834        154   
2               1          9751            12347           834        154   
3               0          4808             6446           693         34   
4               1         10631            14199          1079        373   
...           ...           ...              ...           ...        ...   
27765           0          2672             3874           228        510   
27766           0          1166             1803           865        399   
27767           0          5151             6818           520        163   
27768           0           662             1119           748        247   
27769           0          1251             1945            49        615   

       Место ДТП  Вид извещения о событии  Характер события  \
0           

In [28]:
from sklearn.metrics import recall_score
import numpy as np

# Получение вероятностей принадлежности к классу "мошенничество"
probs = model.predict_proba(X_test)[:, 1]

# Сортировка вероятностей и истинных меток
thresholds = np.sort(probs)
y_true_sorted = y_test[np.argsort(probs)]

for threshold in thresholds:
    y_pred = probs > threshold
    recall = recall_score(y_true_sorted, y_pred)
    if recall <= 0.8:
        break

print(f"Оптимальный порог: {threshold}")


KeyError: '[161, 680, 77, 2679, 2490, 1982, 2842, 1080, 2027, 1020, 1300, 1481, 2467, 1901, 2155, 2644, 2433, 634, 97, 1445, 2491, 1036, 2806, 2729, 480, 694, 1773, 1401, 1089, 2358, 1230, 1308, 1131, 2575, 721, 1423, 2583, 2749, 2489, 1637, 1813, 2010, 617, 470, 1301, 1782, 2017, 2454, 1919, 636, 2263, 2393, 2007, 838, 961, 2307, 128, 780, 2131, 1455, 2752, 81, 1904, 2761, 312, 329, 403, 957, 730, 1071, 767, 1074, 2251, 1125, 344, 567, 2743, 207, 471, 2028, 2514, 2457, 2625, 1859, 814, 1211, 1538, 212, 359, 2383, 976, 391, 1867, 1931, 1988, 585, 758, 190, 191, 1633, 2422, 1241, 2656, 2063, 1140, 853, 1967, 2076, 2797, 418, 1990, 2822, 1550, 796, 2460, 1979, 942, 165, 342, 912, 2810, 1521, 591, 678, 2689, 1110, 183, 66, 905, 748, 972, 1216, 227, 1400, 1541, 1959, 2637, 818, 1719, 1780, 2787, 2195, 1917, 1268, 2599, 2709, 728, 1587, 953, 1661, 2293, 2445, 1254, 2688, 594, 301, 2776, 1841, 285, 661, 1409, 1456, 1873, 2340, 1668, 2313, 491, 1105, 1877, 2577, 829, 1651, 2006, 2720, 826, 23

In [15]:
# Обратное преобразование столбца 'тип убытка'
# insurance_data_encoded['тип убытка'] = label_encoders['тип убытка'].inverse_transform(insurance_data_encoded['тип убытка'])
# insurance_data_encoded['тип убытка']

# Обучение модели Random Forest

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Разделение данных
X = insurance_data_encoded.drop('мошенник', axis=1)
y = insurance_data_encoded['мошенник']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Выбор модели
model = RandomForestClassifier()

# Обучение модели
model.fit(X_train, y_train)

# Оценка модели
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      2802
           1       1.00      0.34      0.51        97

    accuracy                           0.98      2899
   macro avg       0.99      0.67      0.75      2899
weighted avg       0.98      0.98      0.97      2899



# Проверка на сгенерированных данных

В этом разделе происходит проверка на новых, сгенерированных данных. Данные были подготовлены и закодированы таким же образом, как и обучающие данные. Затем обученная модель использовалась для предсказания на этих новых данных.

Предсказание представляет собой бинарное значение:
- **0** означает, что, согласно модели, это **не мошенник**.
- **1** означает, что, согласно модели, это **мошенник**.

```python
predictions = model.predict(new_data_encoded)
print(predictions)


In [17]:
# Генерация новых данных
new_data = pd.DataFrame({
    'тип убытка': ['У'],
    'Номер убытка': ['У-000-01799744/16'],
    'Номер претензии': ['У-000-01799744/16/3'],
    'Дата события': ['20.06.2012'],
    'Время ДТП': ['13:00:00'],
    'Место ДТП': ['Автомагистраль'],
    'Вид извещения о событии': ['Заявление'],
    'Характер события': ['ДТП с участием 2-х ТС с пострадавшими'],
    'Оформлено сотрудниками ГИБДД': ['ДА'],
    'Адрес ДТП: Страна': ['Россия'],
    'Дата регистрации претензии': ['29.11.2016'],
    'Договор': ['ВВВ-0603579000'],
    'Риск': ['Риски ОСАГО'],
    'Объект': ['Man TGA'],
    'Основание для претензии': ['Судебное решение'],
    'Характер ущерба': ['Смерть в результате ДТП'],
    'Форма возмещения': ['Денежная'],
    'Основание для регресса': ['Алкогольное, наркотическое или иное токсическое опьянение, отказ от прохождении медицинского освидетельствования на состояние опьянения'],
    'Дата предоставления полного пакета документов': ['19.10.2016'],
    'сумма': [12500.0],
    'проверка': 0
})

# Кодирование новых данных таким же образом, как и обучающие данные
new_data_encoded = new_data.copy()
for col in new_data.columns:
    if new_data[col].dtype == 'object' or new_data[col].dtype == 'bool':
        new_data_encoded[col] = label_encoders[col].transform(new_data_encoded[col].astype(str))

# Предсказание с использованием обученной модели
predictions = model.predict(new_data_encoded)
print(predictions)

[0]


# Обучение модели Gradient Boosting

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

# Разделение данных
X = insurance_data_encoded.drop('мошенник', axis=1)
y = insurance_data_encoded['мошенник']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Выбор модели
model = GradientBoostingClassifier()

# Обучение модели
model.fit(X_train, y_train)

# Оценка модели
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98      2802
           1       0.62      0.08      0.15        97

    accuracy                           0.97      2899
   macro avg       0.79      0.54      0.56      2899
weighted avg       0.96      0.97      0.96      2899

